<a href="https://colab.research.google.com/github/Xzane-09/SBD/blob/Temp/VH_Automation_gcolab_version16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the libraries

In [ ]:
#!pip install --upgrade openpyxl
#!pip uninstall openpyxl

In [ ]:
import sys
import subprocess

# List of libraries:
list_lib = ['googletrans==3.1.0a0','python-Levenshtein','fuzzywuzzy','gTTS']
upd_lib = ['openpyxl']

for lib in upd_lib:
  subprocess.check_call([sys.executable, '-m', 'pip', 'install','--upgrade', lib])

# implement pip as a subprocess:
for lib in list_lib:
  subprocess.check_call([sys.executable, '-m', 'pip', 'install',lib])

# process output with an API in the subprocess module:
reqs = subprocess.check_output([sys.executable, '-m', 'pip',
'freeze'])

for lib in list_lib:
  installed_packages = [r.decode().split('==')[0] for r in reqs.split()]
  if lib in installed_packages:
    print(lib)

for lib in upd_lib:
  installed_packages = [r.decode().split('==')[0] for r in reqs.split()]
  if lib in installed_packages:
    print(lib)



python-Levenshtein
fuzzywuzzy
gTTS
openpyxl


## Import the libraries

In [ ]:
#  ========================================= Basic Imports ====================================================================== 
import pandas as pd
import os
import io
import sys
from datetime import datetime, timedelta
import re
import glob
import numpy as np
import ssl
import warnings
from google.colab import drive
from google.colab import files
import chardet

warnings.filterwarnings("ignore")

#========================================= Processing Bar notification ====================================================
from googletrans  import Translator
translator = Translator(service_urls = ['translate.googleapis.com']) 

#========================================= Processing Bar notification ====================================================
from tqdm.notebook import tqdm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#========================================= Model Imports ==================================================================
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler,MinMaxScaler,MaxAbsScaler

#========================================= For sound notification =========================================================
from gtts import gTTS

    

In [ ]:
process_start_time =  datetime.now()
scaling = MaxAbsScaler
tfidf_transformer = TfidfTransformer()
count_vect = CountVectorizer()
SGD = SGDClassifier(loss='hinge', penalty='l2', max_iter=1000, learning_rate='constant',eta0 = 0.1)



### All definations

In [ ]:
def reading_master_files():
  uploads = files.upload()
  tot_files = len(uploads.keys())

  if tot_files>1:
    frame=[]
    total = 0
    for fn in uploads.keys():
        file = pd.read_excel(io.BytesIO(uploads[fn]))
        rep_col = 'Predicted Names/ Model Cleansed'
        rep_col_dict = {'Predicted Names/ Model Cleansed':'New Supplier'} # new supplier as manually harmonized names/previously harmonized/model cleansed
        for col in file.columns:
            if col == rep_col:
                file.rename(columns = rep_col_dict, inplace = True)
        file = file[['DataSource','Vendor number','Vendor name', 'New Supplier']]
        dimen = file.shape
        nr = dimen[0]
        nc = dimen[1]
        total=total+nr
        file.reset_index(inplace=True,drop=True)
        frame.append(file)
    consolidated_df = pd.concat(frame,sort=False)
    consolidated_df.reset_index(inplace=True,drop=True)
    raw_df = consolidated_df.copy()

    for fn in uploads.keys():
      with open(fn, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(18))
        print(f'Filename: {fn}\nInfo: {result} \n')

  else:
    filename = uploads.keys()
    filename = str(filename)
    filename = re.findall(r"\'.*?\'", filename)[0]
    filename = re.sub("'","",filename)
    with open(filename, 'rb') as rawdata:
      result = chardet.detect(rawdata.read(18))
      print(f'Filename: {filename}\nInfo: {result} \n')
    file = pd.read_excel(io.BytesIO(uploads[filename]))
    dimen = file.shape
    nr = dimen[0]
    nc = dimen[1]
    total=nr
    file = file[['DataSource','Vendor number','Vendor name', 'New Supplier']]
    raw_df = file.copy()

  return(raw_df,total,tot_files)


def vendor_harmonized_stop_word(name):
    s = " ".join([i for i in re.split(r' ', name) if not i in vendor_harmonized_stopwords]).strip()
    return s


def trim_spaces(name):
#    s = " ".join([i for i in re.split(r' ', name)]).strip()
#    return s
    s = " ".join(name.split()).strip()
    return s


def new_old_match_stopword(name):
    s = "".join([i for i in re.split(r' ', name) if not i in new_old_match_stopwords]).strip()
    return s


def Clean_Vendor_Name_stopword(name):
    s = " ".join([i for i in re.split(r' ', name) if not i in new_old_match_stopwords]).strip()
    return s

    

def rep(name):
    return(name.replace("`","",1))

def data_clean(file):
    proc_df = file
    ## Try to implement try catch if there are. No major changes are implemented in 
    ## Clean_Vendor_name. Only remove non-alpha-numeric names/character. Will be considering 
    ## Clean_Vendor_name as Train_X and will later mapped to original vendor name.
    ## Also in put file the incoming vendor names should also be cleaned.
    proc_df['Clean_Vendor_Name'] = proc_df['Vendor name'].apply(lambda x: str(x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: x.replace('DO NOT USE', ''))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: re.sub('[#,.,/,]', ' ',x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: x.lower())
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(vendor_harmonized_stop_word)
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(Clean_Vendor_Name_stopword) # added this new line
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(trim_spaces) # added this new line
    
    return proc_df


def data_clean_initials(file):
    proc_df = file
    proc_df['Vendor_name_Initials'] = proc_df['Clean_Vendor_Name'].apply(lambda x: str(x))
    #proc_df['Vendor_name_Initials'] = proc_df['Vendor_name_Initials'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    #print()
    proc_df['Vendor_name_Initials'] = proc_df['Vendor_name_Initials'].apply(lambda x: x[0])
    proc_df['Vendor_name_Initials'] = proc_df['Vendor_name_Initials'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    proc_df['Vendor_name_Initials'] = proc_df['Vendor_name_Initials'].apply(lambda x: re.sub('[@,?,#,(,),*,+,.,-,_,`,~]', ' ',x))
    #proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(vendor_harmonized_stop_word)
    
    proc_df['New_Supplier'] = proc_df['New Supplier'].apply(lambda x: str(x))
    proc_df['New_Supplier'] = proc_df['New Supplier'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    proc_df['New_Supplier'] = proc_df['New_Supplier'].apply(lambda x: re.sub('[@?#()*_~]', '',x))
    proc_df['New_Supplier'] = proc_df['New_Supplier'].apply(lambda x: x.lower())
    proc_df['New_Supplier'] = proc_df['New_Supplier'].apply(lambda x: re.sub('"', '',x))
    proc_df['New_Supplier'] = proc_df['New_Supplier'].apply(lambda x: re.sub("'", '',x))
    #proc_df['New_Supplier'] = proc_df['New_Supplier'].apply(lambda x: re.sub(r'-', ' ', x))
    #proc_df['New_Supplier'] = proc_df['New_Supplier'].apply(lambda x: re.sub(r'[^\s]', '', x))
    proc_df['New_Supplier'] = proc_df['New_Supplier'].apply(vendor_harmonized_stop_word) # why we are removing from new_supplier
    return proc_df


def input_df_data_clean(file):
    proc_df = file
    
    # Below steps are the cleaning for vendors:
    proc_df['Clean_Vendor_Name'] = proc_df['Vendor name'].apply(lambda x: str(x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: re.sub('[#,.,/,]', ' ',x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(vendor_harmonized_stop_word)
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: x.lower())
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(Clean_Vendor_Name_stopword)# added this line extra
    proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(trim_spaces)
    proc_df.replace("", np.nan, inplace=True) #try with commenting this line and below too to check the difference
    proc_df.dropna(subset = ["Clean_Vendor_Name"],inplace=True)
    return proc_df


def crisp_vendor_name(file):
    proc_df = file
    proc_df['crisp_vendor_name'] = proc_df['Clean_Vendor_Name'].apply(lambda x: str(x))
    #proc_df['crisp_vendor_name'] = proc_df['crisp_vendor_name'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    #proc_df['crisp_vendor_name'] = proc_df['crisp_vendor_name'].apply(lambda x: re.sub(' ', '',x))
    proc_df['crisp_vendor_name'] = proc_df['crisp_vendor_name'].apply(new_old_match_stopword)
    return proc_df


   
def input_df_data_ini(file):
    proc_df = file
    proc_df['Vendor_name_Initials'] = proc_df['Clean_Vendor_Name'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    proc_df['Vendor_name_Initials'] = proc_df['Vendor_name_Initials'].apply(lambda x: x[0])
    proc_df['Vendor_name_Initials'] = proc_df['Vendor_name_Initials'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    proc_df['Vendor_name_Initials'] = proc_df['Vendor_name_Initials'].apply(lambda x: re.sub('[@,?,#,(,),*,+,.,-,_,`,~]', ' ',x))
    #proc_df['Clean_Vendor_Name'] = proc_df['Clean_Vendor_Name'].apply(vendor_stop_word)
    return proc_df


def inp_CleaningChamber(file):
    proc_df = file
    proc_df['Vendor_Name_cleansed'] = proc_df['Vendor name'].apply(lambda x: str(x))
    proc_df['Vendor_Name_cleansed'] = proc_df['Vendor_Name_cleansed'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    proc_df['Vendor_Name_cleansed'] = proc_df['Vendor_Name_cleansed'].apply(lambda x: re.sub('[@?#()*_~/]', ' ',x))
    proc_df['Vendor_Name_cleansed'] = proc_df['Vendor_Name_cleansed'].apply(lambda x: x.lower())
    proc_df['Vendor_Name_cleansed'] = proc_df['Vendor_Name_cleansed'].apply(lambda x: re.sub('"', '',x))
    #proc_df['Vendor_Name_cleansed'] = proc_df['Vendor_Name_cleansed'].apply(lambda x: re.sub("'", '',x))
    #proc_df['Vendor_Name_cleansed'] = proc_df['Vendor_Name_cleansed'].apply(lambda x: re.sub(r'-', ' ', x))
    #proc_df['New_Supplier'] = proc_df['New_Supplier'].apply(lambda x: re.sub(r'[^\s]', '', x))
    proc_df['Vendor_Name_cleansed'] = proc_df['Vendor_Name_cleansed'].apply(vendor_harmonized_stop_word)
    proc_df['Vendor_Name_cleansed'] = proc_df['Vendor_Name_cleansed'].apply(trim_spaces)
    return proc_df


def bucket_df(df,initials_lists):
    bucket_df_list =[]
    grp_df = df.groupby(df.Vendor_name_Initials)
    for char in initials_lists:
        bucket_df_list.append(grp_df.get_group(char))
    return(bucket_df_list)



def time_util(pred_time):
    # total_seconds = pred_time.seconds
    seconds = int(pred_time.total_seconds())
    secs_in_a_min = 60
    minutes, seconds = divmod(seconds, secs_in_a_min)

    time_fmt = f"{minutes:02d} {seconds:02d} secs"

    if minutes > 0:
        suffix = 's' if minutes > 1 else ""
        return f'{minutes} min{suffix} & {seconds} secs'

    return time_fmt


def sgd_clf(i):
    Tot_St_Time = datetime.now()
    grp_number = np.unique(grp_clean_main_df_list[i]['Vendor_name_Initials'])[0]
    Train_X,Test_X,Train_Y,Test_Y = train_test_split(grp_clean_main_df_list[i]['Clean_Vendor_Name'],grp_clean_main_df_list[i]['New_Supplier'],test_size=0.00001, random_state=0)
    X_train_counts = count_vect.fit_transform(Train_X)
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    sgd_clf=SGD.fit(X_train_tfidf,Train_Y)
    Tot_End_Time = datetime.now()
    Tot_time_taken = Tot_End_Time - Tot_St_Time
    return(time_util(Tot_time_taken),grp_number,sgd_clf)

def Match_wordLength(i):
    if (merged_final['Model_pred_Supplier'][i] == merged_final['Prev_Predicted_Supplier'][i]):
        merged_final['Match_word_len'].loc[i] = 'Same Length'
    else:
        merged_final['Match_word_len'].loc[i] = 'Different Length'
    return(merged_final['Match_word_len'])

def rep(name):
    return(name.replace("-","",1))


def trans_name(name):
  srch = re.search("([^\x00-\x7F])+",name)==None
  if srch:
    return(name)
  else:
    return(translator.translate(str(name), dest = 'en').text)




punc = '''!(-[](){};`:)/'"\,<>.?@#$%^*+_~'''  
invalid_punc = ['*','#']
def rem_first_char_punc(name):  # check this whether it is implemented or not
    count = 0
    frame=[]
    for i in range(len(name)):
        if name[i] in punc:
            frame.append(name[i])
            #print(name[i])
            count = count+1
    unq_name_punc = np.unique(frame)
    unq_name_punc = unq_name_punc.tolist()
    for i in range(len(invalid_punc)):
        if invalid_punc[i] in unq_name_punc:
            name=name.replace(invalid_punc[i],'')

    limit = 0
    for i in range(len(name)):
        if name[i] not in punc:
            #print(name[i])
            break
        else:
            #print(name[i])
            limit = limit + 1
            #break
    upper_limit = limit  
    #print(upper_limit) 
    for i in range(upper_limit):
        if name[0] in punc:
            #print(f'the found punctuation is "{name[0]}"')
            name=name.replace(name[0],'',1)
    return(name)


def ds_cleansing(name):
  #print(type(name))
  if type(name) is np.str:
    punc_list = [',','.']
    for c in name:
        if c not in punc_list:
            pass
        else:
            split_string = re.split(f"[{c}]", name)
            name = split_string[len(split_string)-1].strip()
  else:
    pass
  return(name)


def rep_harmonized_sup_name(name):
  #name = str(name)
  for i in re.split(r' ', name):
    if str.upper(i) in supplier_stopwords:
      idx = name.index(i)
      if idx == 0:
        return(name.replace(i,"",1))
    else:
      return(name)


def final_output_sup_name_Cleaning(file,column_name):
    proc_df = file.copy()
    proc_df[column_name]= proc_df[column_name].apply(rep_harmonized_sup_name)
    return(proc_df)



def vendorFlag_wo_VN(i):
    #if (input_df['Clean_Vendor_Name'].iloc[i] in clean_main_df['Clean_Vendor_Name']):
    if (input_df['Clean_Vendor_Name'][i] in clean_main_df['Clean_Vendor_Name'].values)|(input_df['crisp_vendor_name'][i] in clean_main_df['crisp_vendor_name'].values):
      input_df['Vendor_Flag'].iloc[i] = 'Old'
    #elif (input_df['crisp_Vendor_Name'][i] in clean_main_df['crisp_Vendor_Name'].values):
    #  input_df['Vendor_Flag'].iloc[i] = 'Old'
    else:
      input_df['Vendor_Flag'].iloc[i] = 'New'
    return(input_df)



def vendorFlag(i):
    #if (input_df['Clean_Vendor_Name'].iloc[i] in clean_main_df['Clean_Vendor_Name']):
    if ((input_df['Clean_Vendor_Name'][i] in clean_main_df['Clean_Vendor_Name'].values) and\
        (input_df['DataSource'][i] in clean_main_df['DataSource'].values) and\
        (input_df['Vendor number'][i] in clean_main_df['Vendor number'].values))\
    or ((input_df['crisp_vendor_name'][i] in clean_main_df['crisp_vendor_name'].values) and (input_df['DataSource'][i] in clean_main_df['DataSource'].values) and (input_df['Vendor number'][i] in clean_main_df['Vendor number'].values)):
      #if(input_df['DataSource'][i] in clean_df['DataSource'].values) and (input_df['Vendor number'][i] in clean_df['Vendor number'].values):
      input_df['Vendor_Flag'].iloc[i] = 'Old'
    #elif (input_df['crisp_Vendor_Name'][i] in clean_main_df['crisp_Vendor_Name'].values):
    #  input_df['Vendor_Flag'].iloc[i] = 'Old'
    else:
      input_df['Vendor_Flag'].iloc[i] = 'New'
    return(input_df)


def lingual_check(file):
  proc_df = file
  for n in proc_df[proc_df.columns[0]]:
#  #print(n)
    srch = re.search("([^\x00-\x7F])+",n)==None
    if srch:
      print('no match')
      break
    else:
      print('There are other format of languages present in the file')
      break


In [ ]:
master_file,length,no_of_files = reading_master_files()

Saving Master data.xlsx to Master data.xlsx
Filename: Master data.xlsx
Info: {'encoding': None, 'confidence': 0.0, 'language': None} 



In [ ]:
#upd_lib = []
#try:
#  master_file,length,no_of_files = reading_master_files()
#except ImportError as ie:
#  error_args = ie.args
#  error_msg = ie.msg
#  #print(ie.error_msg)
#  error_msg = error_msg.lower()
  
#  if 'requires version' in error_msg:
#    match = re.search(r"[\d.]+",error_msg) 
#    if match != None: 
#      ver_number = match.group(0)
#      temp_lib = re.search(r"'[(A-Za-z)']+",error_msg)
#      temp_lib = temp_lib.group(0)
#      temp_lib = temp_lib.replace("'","")
#      if temp_lib not in upd_lib:
#        upd_lib.append(temp_lib)
#        for lib in upd_lib:
#          subprocess.check_call([sys.executable, '-m', 'pip', 'install','--upgrade', lib])
#      
#      reqs = subprocess.check_output([sys.executable, '-m', 'pip','freeze'])

#      for lib in upd_lib:
#        installed_packages = [r.decode().split('==')[0] for r in reqs.split()]
#        if lib in installed_packages:
#          print(lib)

  

In [ ]:
print('User uploaded master file with {length} records'.format(length=length))

User uploaded master file with 228989 records


In [ ]:
# Master file
master_file.head(4)

,DataSource,Vendor number,Vendor name,New Supplier
0,68. 3rd Party Security,6412,� Leeuwenburgh Mark,LEEUWENBURGH MARK
1,10. SAP C11,30117052,"""ANTIESTATICA DE MEXICO, SA de CV""",ANTIESTATICA DE MEXICO
2,15. SAP SP1,102156,"""Le Bleu, Phillip""",LE BLEU PHILLIP
3,12. SAP SEP,110302,# 3W # DO NOT USE,MICROSOFT


In [ ]:
raw_df = master_file.copy()
raw_df['Vendor name'].drop_duplicates(inplace=True)
raw_df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228989 entries, 0 to 228988
Columns: 4 entries, DataSource to New Supplier
dtypes: object(4)
memory usage: 7.0+ MB


#### Dropping the null rows:

In [ ]:
# Use raw_df to get vendor number and DataSource at the end. So copy only Vendor name and new supplier to df
df = raw_df.copy()
df.dropna(inplace=True)
df['Vendor name'].drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228922 entries, 0 to 228921
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   DataSource     228922 non-null  object
 1   Vendor number  228922 non-null  object
 2   Vendor name    228922 non-null  object
 3   New Supplier   228922 non-null  object
dtypes: object(4)
memory usage: 8.7+ MB


In [ ]:
after_drop_na_1 = df.shape[0]

In [ ]:
no_of_null_records1= length - after_drop_na_1
no_of_null_records1

67

In [ ]:
df['DataSource']=df['DataSource'].apply(ds_cleansing)

In [ ]:
df.sample(2)

,DataSource,Vendor number,Vendor name,New Supplier
204971,SAP P10,1107062,TREASURER OF STATE OF NJ/DBA DEPT OF COMM AFFAIRS,TREASURER
155635,New Ariba,1000016668,PRECOIN S L,PRECOIN S L


### Defined set of stopwords:

In [ ]:
vendor_harmonized_stopwords = ['me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
                "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
                'yourselves', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
                'herself', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', #'it',
                'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these',
                'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has',
                'had', 'having', 'do', 'does', 'did', 'doing','an', #'the', 'and',
                'but','if', 'or', 'because', 'until', 'while', 'of', 'at', 'by', 'with', #'for', 
                'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
                'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
                'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where',
                'why', 'how', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
                'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
                'very', 'can', 'just', 'don', "don't", 'should', "should've",
                'now', 'll', 're', 've', 'ain', 'aren', "aren't", 'couldn',
                "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
                "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't",
                'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
                'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't",'dr','use' #'co',
                #'company', 'ltd', 'co ltd', 'company ltd', 'inc', 'llc', 'lp', 'corp', 'corporation',
                ]#'a','d', 's','i','m', 't', 'o', 'y','he','all','will', 'as'



In [ ]:
new_old_match_stopwords = ['co', 'company', 'ltd', 'co ltd', 'company ltd', 'inc', 'llc', 'lp', 'corp', 'corporation','ltl','ftl']

In [ ]:
supplier_stopwords = ['ZHEJIANG','ZHANGZHOU','ZHONGSHAN','QINGDAO','QIDONG','SUZHOU'] # all words should be in upper case

In [ ]:
df['Vendor name'] = df['Vendor name'].apply(rem_first_char_punc)  # try to implement for input data also IF NOT
df['New Supplier'] = df['New Supplier'].apply(rem_first_char_punc)
df['Vendor name'].drop_duplicates(inplace=True) 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228922 entries, 0 to 228921
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   DataSource     228922 non-null  object
 1   Vendor number  228922 non-null  object
 2   Vendor name    228922 non-null  object
 3   New Supplier   228922 non-null  object
dtypes: object(4)
memory usage: 8.7+ MB


In [ ]:
sh_df = df.shape[0]
sh_df

228922

In [ ]:
# Below code is to verify whether are there any names apart from english lingual
count = 0
for n in df['New Supplier']:
  srch = re.search("([^\x00-\x7F])+",n)==None
  if srch:
    pass
  else:
    count = count + 1
print(f'Foreign name formats in "new supplier" column : {count}')



count = 0
for n in df['Vendor name']:
  srch = re.search("([^\x00-\x7F])+",n)==None
  if srch:
    pass
  else:
    count = count + 1
print(f'Foreign name formats in "Vendor name" column : {count}')


Foreign name formats in "new supplier" column : 3535
Foreign name formats in "Vendor name" column : 7461


In [ ]:
df.head(2)

,DataSource,Vendor number,Vendor name,New Supplier
0,3rd Party Security,6412,� Leeuwenburgh Mark,LEEUWENBURGH MARK
1,SAP C11,30117052,"ANTIESTATICA DE MEXICO, SA de CV""",ANTIESTATICA DE MEXICO


In [ ]:
## This snippet will print the name of the Column IF there are any NULL records in that Column
if (df.isna().sum().unique()[0] != 0):
  for i in df.columns:
    if df[i].isna().sum() !=0:
      print(i + '\n')
    else:
      pass
else:
  pass

In [ ]:
clean_df = data_clean(df)
clean_df['Clean_Vendor_Name'] = clean_df['Clean_Vendor_Name'].apply(trim_spaces)
clean_df.replace("", np.nan, inplace=True)
clean_df.dropna(subset = ["Clean_Vendor_Name","Vendor name"],inplace=True)
clean_df = data_clean_initials(clean_df)
clean_df = crisp_vendor_name(clean_df)

clean_df['DataSource'] = clean_df['DataSource'].apply(lambda x: str(x))
clean_df['Vendor number'] = clean_df['Vendor number'].apply(lambda x: str(x))
clean_df['DataSource'] = clean_df['DataSource'].apply(trim_spaces)
clean_df['Vendor number'] = clean_df['Vendor number'].apply(trim_spaces)


clean_df= clean_df[['DataSource', 'Vendor number', 'Vendor name','Clean_Vendor_Name','crisp_vendor_name','New Supplier','New_Supplier','Vendor_name_Initials']]
clean_df.reset_index(inplace=True,drop=True)

In [ ]:
clean_df.sample(2)

,DataSource,Vendor number,Vendor name,Clean_Vendor_Name,crisp_vendor_name,New Supplier,New_Supplier,Vendor_name_Initials
145846,SAP SEP,104437,ORANGE,orange,orange,ORANGE BUSINESS SERVICES,orange business services,o
58798,3rd Party Security,40000480,DUSTIN SVERIGE AB,dustin sverige ab,dustinsverigeab,DUSTIN SVERIGE,dustin sverige,d


In [ ]:
#clean_df.info()

In [ ]:
sh_clean_df = clean_df.shape[0]

In [ ]:
no_of_null_records2 = sh_df - clean_df.shape[0]
#no_of_null_records2

In [ ]:
clean_df.replace("", np.nan, inplace=True)
clean_df.dropna(subset = ["Clean_Vendor_Name"],inplace=True)
clean_df.dropna(subset = ['New_Supplier'],inplace=True)
clean_ini_list = clean_df['Vendor_name_Initials'].apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x)).unique()
clean_ini_list = np.sort(clean_ini_list)
clean_ini_list = list(filter(str.strip, clean_ini_list))

In [ ]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225459 entries, 0 to 225929
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   DataSource            225459 non-null  object
 1   Vendor number         225459 non-null  object
 2   Vendor name           225459 non-null  object
 3   Clean_Vendor_Name     225459 non-null  object
 4   crisp_vendor_name     225459 non-null  object
 5   New Supplier          225459 non-null  object
 6   New_Supplier          225459 non-null  object
 7   Vendor_name_Initials  225459 non-null  object
dtypes: object(8)
memory usage: 15.5+ MB


In [ ]:
no_of_null_records3 = sh_clean_df - clean_df.shape[0]
#no_of_null_records3

In [ ]:
# Filtering the Clean data from noisy data with Non-English Vendor Names
clean_df['noise_flag'] = clean_df['Vendor name'].apply(lambda x: True if (re.findall("([^\x00-\x7F])+",x)) else False)
clean_main_df = clean_df[clean_df['noise_flag'] == False]
clean_main_df.reset_index(inplace=True,drop=True)

In [ ]:
clean_main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220956 entries, 0 to 220955
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   DataSource            220956 non-null  object
 1   Vendor number         220956 non-null  object
 2   Vendor name           220956 non-null  object
 3   Clean_Vendor_Name     220956 non-null  object
 4   crisp_vendor_name     220956 non-null  object
 5   New Supplier          220956 non-null  object
 6   New_Supplier          220956 non-null  object
 7   Vendor_name_Initials  220956 non-null  object
 8   noise_flag            220956 non-null  bool  
dtypes: bool(1), object(8)
memory usage: 13.7+ MB


In [ ]:
sh_clean_main_df = clean_main_df.shape[0]
#sh_clean_main_df

In [ ]:
no_of_noise_flags = clean_df.shape[0] - clean_main_df.shape[0]
#no_of_noise_flags

In [ ]:
clean_main_df.head(2)

,DataSource,Vendor number,Vendor name,Clean_Vendor_Name,crisp_vendor_name,New Supplier,New_Supplier,Vendor_name_Initials,noise_flag
0,SAP C11,30117052,"ANTIESTATICA DE MEXICO, SA de CV""",antiestatica de mexico sa de cv,antiestaticademexicosadecv,ANTIESTATICA DE MEXICO,antiestatica de mexico,a,False
1,SAP SP1,102156,"Le Bleu, Phillip""",le bleu phillip,lebleuphillip,LE BLEU PHILLIP,le bleu phillip,l,False


In [ ]:
tot_no_of_clean_records = clean_main_df.shape[0]
#tot_no_of_clean_records

In [ ]:
rest_main_df = clean_df[clean_df['noise_flag'] == True]
rest_main_df.reset_index(inplace=True,drop=True)

In [ ]:
rest_main_df.shape[0] # Duplicates may be present. So check the count of 'no_of_unique_dirty_records'

4503

In [ ]:
rest_main_df.head(2)

,DataSource,Vendor number,Vendor name,Clean_Vendor_Name,crisp_vendor_name,New Supplier,New_Supplier,Vendor_name_Initials,noise_flag
0,3rd Party Security,6412,� Leeuwenburgh Mark,leeuwenburgh mark,leeuwenburghmark,LEEUWENBURGH MARK,leeuwenburgh mark,l,True
1,SAP E03,151812,주) Lee Gun Tech / Person in Charge: Oh Seok Ha...,lee gun tech person charge oh seok hak wh,leeguntechpersonchargeohseokhakwh,LEE GUN TECH / PERSON IN CHARGE: OH SEOK HAK,lee gun tech / person charge: oh seok hak,l,True


In [ ]:
rest_main_df.drop_duplicates(subset=['Vendor name'], inplace=True)


In [ ]:
no_of_unique_dirty_records = rest_main_df.shape[0] # Unique no of dirty records
no_of_unique_dirty_records

2113

In [ ]:
rest_main_df = rest_main_df[['DataSource', 'Vendor number','Vendor name', 'New Supplier']]

In [ ]:
#This percentage is calculated to know how much irregular unique records are present. Need to check this evertime.
# This percentage is good to be in range of 5~8%
per_irr = format((rest_main_df.shape[0]/clean_df.shape[0])*100, ".3f") # show this to Christian.
if float(per_irr)<8:
    print(f'The percentage of irregular records is below the threshold value (5~8%):  {per_irr}'+'%')
else:
    print(f'The percentage of irregular records is above the threshold value (5~8%):  {per_irr}'+'%')

The percentage of irregular records is below the threshold value (5~8%):  0.937%


## Extracting Bad/uncleaned Master Data: 

In [ ]:
rest_main_df.to_excel('Uncleaned_MainData.xlsx',index=False,encoding='utf8')
files.download('Uncleaned_MainData.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Bucketing the master data/ Group the initials

In [ ]:
grp_clean_main_df_list = bucket_df(clean_main_df,clean_ini_list)

# Input file:

In [ ]:
input_uploads = files.upload()

Saving Percentage of Spend with all columns.xlsx to Percentage of Spend with all columns.xlsx


In [ ]:
# Input file extension validation

name_ext_frame = []
for fn in input_uploads.keys():
  name, extension = os.path.splitext(fn)
  ext = extension.replace('.','')
  name_ext_frame.append([name,ext])


for i in range(len(name_ext_frame)):
    if str.lower(name_ext_frame[i][1]) != 'xlsx':
      print('Alert Info:: file with other extension apart from "XLSX":', name_ext_frame[i])
    else:
      print('Info::'+'\n'+
            'File extension:: XLSX')




Info::
File extension:: XLSX


In [ ]:
tot_files = len(input_uploads.keys())
tot_files

1

In [ ]:
if tot_files >1:
    inp_frame=[]
    #total = 0
    inp_total = 0
    for fn in input_uploads.keys():
        print(f'Filename: {fn}')
        file = pd.read_excel(io.BytesIO(input_uploads[fn]))
        dimen = file.shape
        nr = dimen[0]
        nc = dimen[1]
        inp_total=inp_total+nr
        print(f'\nNo. of Rows: {nr}\nNo. of cols: {nc}\n')
        print('\n')
        
        file.reset_index(inplace=True,drop=True)
        inp_frame.append(file)
    cons_inp_df = pd.concat(inp_frame,sort=False)
    print(f'total: {inp_total}\n___________________________________________________________________________________________')
    input_raw_df = cons_inp_df.copy()
    refined_inp_col_names = (input_raw_df.columns.to_series(index=range(len(input_raw_df.columns))).apply(lambda x : trim_spaces(x))).to_list()
    input_raw_df.columns=refined_inp_col_names
else:
    print('Else- section')
    inp_total = 0
    print(f'Filename: {fn}')
    file = pd.read_excel(io.BytesIO(input_uploads[fn]))
    dimen = file.shape
    nr = dimen[0]
    nc = dimen[1]
    inp_total=inp_total+nr
    print(f'\nNo. of Rows: {nr}\nNo. of cols: {nc}\n')
    print('\n')
    print(f'total: {inp_total}\n_________________________________________________________________________________________________')
    input_raw_df = file.copy()
    refined_inp_col_names = (input_raw_df.columns.to_series(index=range(len(input_raw_df.columns))).apply(lambda x : trim_spaces(str(x)))).to_list()
    input_raw_df.columns=refined_inp_col_names


Else- section
Filename: Percentage of Spend with all columns.xlsx

No. of Rows: 76702
No. of cols: 14



total: 76702
_________________________________________________________________________________________________


In [ ]:
no_of_raw_inp_records = inp_total
no_of_raw_inp_records

76702

In [ ]:
#input_raw_df.head(2)

In [ ]:
# Drop unwanted column names by giving the list of column names in  a list
def drop_cols_df(col_list,dataframe):
  for col in dataframe.columns:
    if col not in col_list:
      dataframe.drop(columns=col,axis=1,inplace=True)
                

# For checking whether there are any duplicates or not

def dup_count(check_dataframe):
  dup_count= check_dataframe[check_dataframe.duplicated()==True].count().unique()[0]
  dup_dataframe = check_dataframe[check_dataframe.duplicated()==True]

  return(dup_count,dup_dataframe)


def check_dup_count(check_df):
  count_dup, dup_df = dup_count(check_df)
  #globals()['count_duplicates'] = count_dup
  if count_dup > 1 and count_dup < 8:
      print(dup_df)
  else:
    print('Total Duplicates in Dataset: ',count_dup)

  return(count_dup)


# To count foreign names
def count_foreign(df,col_name):
  count = 0
  for n in df[col_name]:
    srch = re.search("([^\x00-\x7F])+",n)==None
    if srch:
      pass
    else:
      count = count + 1
  #print(f'Other name formats in "new supplier" column : {count}')
  return(count)



In [ ]:
# give the input column names that is required and need to drop the rest 
acp_inp_col_names = ['DataSource','Vendor number','Vendor name','Spend (USD)','Segment','Primary Category'] 


In [ ]:
drop_cols_df(acp_inp_col_names,input_raw_df)

In [ ]:
input_raw_df.columns

Index(['DataSource', 'Vendor number', 'Vendor name', 'Spend (USD)', 'Segment',
       'Primary Category'],
      dtype='object')

In [ ]:
count_dup = check_dup_count(input_raw_df)

Total Duplicates in Dataset:  0


In [ ]:
#input_raw_df.isna().sum().max()>0

In [ ]:
# Drop Na's and duplicates if present
input_raw_df.dropna(inplace=True)
if count_dup>0:
  input_raw_df.drop_duplicates(inplace=True)
input_raw_df.reset_index(inplace=True,drop=True)

In [ ]:
check_dup_count(input_raw_df)

Total Duplicates in Dataset:  0


0

In [ ]:
input_raw_df.sort_index(inplace=True)
input_raw_df.reset_index(drop=True, inplace=True)

In [ ]:
input_raw_df.shape[0]

76522

In [ ]:
no_of_null_inp_records1 = inp_total - input_raw_df.shape[0]
no_of_null_inp_records1

180

In [ ]:
foreign_count = count_foreign(input_raw_df,'Vendor name')
print(f'Foreign name/names count: {foreign_count}') # if foreign counts greater than 1 then fetch the records by implementing noise flag


Foreign name/names count: 926


In [ ]:
if foreign_count>0:
  #input_raw_df['Vendor name'] = input_raw_df['Vendor name'].apply(trans_name)
  input_raw_df['noise_flag'] = input_raw_df['Vendor name'].apply(lambda x: True if (re.findall("([^\x00-\x7F])+",x)) else False)
  foreign_inp_df = input_raw_df[input_raw_df['noise_flag'] == True]
  foreign_inp_df.to_excel('foreign input data.xlsx',index=False,encoding='utf8')
  files.download('foreign input data.xlsx')
  
  # Also filter non foreign data from input data
  input_raw_df = input_raw_df[input_raw_df['noise_flag'] == False]
  input_raw_df.drop(columns=['noise_flag'],axis = 1, inplace=True)
  input_raw_df.reset_index(drop=True,inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
input_raw_df.shape[0]

75596

In [ ]:
input_raw_df.head(2)

,DataSource,Vendor number,Vendor name,Spend (USD),Segment,Primary Category
0,10. SAP C11,20010276,SAMSUNG SDI CO LTD,3.269116e+08,GTS,Purchased Components
1,10. SAP C11,ICN-5034,STANLEY BLACK AND DECKER CYPRUS/ONE HOLDINGS LTD,2.222926e+08,Undefined,Affiliate


In [ ]:
lingual_check(input_raw_df)

no match


In [ ]:
inp_match_df = input_raw_df.copy()

In [ ]:
inp_match_df.shape

(75596, 6)

In [ ]:
lingual_check(inp_match_df)

no match


In [ ]:
inp_match_df.head(2)

,DataSource,Vendor number,Vendor name,Spend (USD),Segment,Primary Category
0,10. SAP C11,20010276,SAMSUNG SDI CO LTD,3.269116e+08,GTS,Purchased Components
1,10. SAP C11,ICN-5034,STANLEY BLACK AND DECKER CYPRUS/ONE HOLDINGS LTD,2.222926e+08,Undefined,Affiliate


In [ ]:
# Input data cleaning

input_df = input_df_data_clean(input_raw_df)
input_df = input_df_data_ini(input_df)
input_df = crisp_vendor_name(input_df)

# this DS cleaning is optional
for col in input_df.columns:
  if col == 'DataSource':
    input_df['DataSource']=input_df['DataSource'].apply(ds_cleansing)

In [ ]:
records_removed_after_data_cleaning = inp_match_df.shape[0] - input_df.shape[0]
records_removed_after_data_cleaning

5

In [ ]:
input_df.columns

Index(['DataSource', 'Vendor number', 'Vendor name', 'Spend (USD)', 'Segment',
       'Primary Category', 'Clean_Vendor_Name', 'Vendor_name_Initials',
       'crisp_vendor_name'],
      dtype='object')

In [ ]:
for col in input_df.columns:
  if col in ['DataSource','Vendor number']:
    input_df['DataSource'] = input_df['DataSource'].apply(lambda x: str(x))
    input_df['Vendor number'] = input_df['Vendor number'].apply(lambda x: str(x))
    input_df['DataSource'] = input_df['DataSource'].apply(trim_spaces)
    input_df['Vendor number'] = input_df['Vendor number'].apply(trim_spaces)
  else:
    pass


In [ ]:
lingual_check(input_df)

no match


In [ ]:
input_df.shape[0]

75591

In [ ]:
input_df.reset_index(inplace=True,drop=True)

In [ ]:
count_col = 0
for col in input_df.columns:
  if col in acp_inp_col_names:
    count_col = count_col +1
print(count_col)
input_df['Vendor_Flag']=np.nan
if count_col==1:
  for i in tqdm(range(len(input_df))):
    vendorFlag_wo_VN(i)
  input_df['DataSource']=np.nan
  input_df['Vendor number']=np.nan
  
else:
  for i in tqdm(range(len(input_df))):
    vendorFlag(i)



6


  0%|          | 0/75591 [00:00<?, ?it/s]

In [ ]:
input_df.shape[0]

75591

In [ ]:
#stop

In [ ]:
# For test -- 

#input_df = input_df.sample(1000)
#input_df.Vendor_Flag.unique()

In [ ]:
input_df['Vendor name'].nunique()

52200

In [ ]:
input_df.replace(" ", np.nan, inplace=True)

In [ ]:
input_df.shape[0]

75591

In [ ]:
inp_df_shape1 = input_df.shape[0]
inp_df_shape1

75591

In [ ]:
input_df['Vendor_name_Initials'].isna().sum()

0

In [ ]:
input_df.dropna(subset = ["Vendor_name_Initials"],inplace=True) 
# this code can also be alter where one choose to have the filtered input data 

In [ ]:
# dropping NA again IF there is any while creating Vendor_name_Initials else it will be zero
no_of_null_inp_records2 = inp_df_shape1 - input_df.shape[0]
no_of_null_inp_records2

0

In [ ]:
input_df.shape

(75591, 10)

In [ ]:
input_df['Vendor name'].isna().sum()

0

In [ ]:
input_df.reset_index(inplace=True,drop=True)

In [ ]:
#input_df[input_df['Clean_Vendor_Name'] == input_df['Clean_Vendor_Name'].iloc[276]]

In [ ]:
input_df.shape[0]

75591

In [ ]:
n= input_df.Vendor_Flag.nunique()
print(n)

2


In [ ]:
def unique_vendor_flag(n):
  if n==2:
    x,y = input_df.Vendor_Flag.unique()
    return(x,y)
  else:
    return(input_df.Vendor_Flag.unique())

In [ ]:
new_records_with_dup = input_df[input_df.Vendor_Flag =='New'].shape[0] #Duplicate vendor names
new_records_with_dup

12858

In [ ]:
old_records_with_dup = input_df[input_df.Vendor_Flag =='Old'].shape[0] #Duplicate vendor names
old_records_with_dup

62733

In [ ]:
inp_new_flag_df = input_df[input_df.Vendor_Flag =='New']
inp_new_flag_df.reset_index(drop=True,inplace=True)
inp_new_flag_df.head(5)

,DataSource,Vendor number,Vendor name,Spend (USD),Segment,Primary Category,Clean_Vendor_Name,Vendor_name_Initials,crisp_vendor_name,Vendor_Flag
0,SAP C11,ICN-5034,STANLEY BLACK AND DECKER CYPRUS/ONE HOLDINGS LTD,2.222926e+08,Undefined,Affiliate,stanley black and decker cyprus one holdings,s,stanleyblackanddeckercyprusoneholdings,New
1,QAD CH,Y05861,SCOTIABANK,9.596548e+07,GTS,Not Controllable,scotiabank,s,scotiabank,New
2,CASS,UPSF-UPS SUPPLY CHAIN SOLUTIONS IN,UPS SUPPLY CHAIN SOLUTIONS IN,5.659371e+07,Undefined,Transportation,ups supply chain solutions in,u,upssupplychainsolutionsin,New
3,SAP C11,30101603,"Suzhou Botai Human Resources Service Co., Ltd.",4.550125e+07,Undefined,General Procurement,suzhou botai human resources service,s,suzhoubotaihumanresourcesservice,New
4,SAP C11,20018266,MTD SOUTHWEST INC,4.163210e+07,Undefined,Affiliate,mtd southwest,m,mtdsouthwest,New


In [ ]:
new_records = inp_new_flag_df['Vendor name'].nunique()
new_records

10430

In [ ]:
inp_old_flag_df = input_df[input_df.Vendor_Flag == 'Old']
inp_old_flag_df.reset_index(drop=True,inplace=True)
inp_old_flag_df.shape[0]

62733

In [ ]:
old_records = inp_old_flag_df['Vendor name'].nunique()
old_records

42728

In [ ]:
inp_old_flag_df.head()

,DataSource,Vendor number,Vendor name,Spend (USD),Segment,Primary Category,Clean_Vendor_Name,Vendor_name_Initials,crisp_vendor_name,Vendor_Flag
0,SAP C11,20010276,SAMSUNG SDI CO LTD,3.269116e+08,GTS,Purchased Components,samsung sdi,s,samsungsdi,Old
1,CASS,KHNN-KUEHNE NAGEL SVCS LTD,KUEHNE NAGEL SVCS LTD,2.065791e+08,Undefined,Transportation,kuehne nagel svcs,k,kuehnenagelsvcs,Old
2,SAP C11,20005014,CHANG TYPE INDUSTRIAL CO. LTD.,1.917764e+08,GTS,Finished Goods,chang type industrial,c,changtypeindustrial,Old
3,SAP C11,20012882,DET LOGISTICS USA CORPORATION,9.484559e+07,GTS,Purchased Components,det logistics usa,d,detlogisticsusa,Old
4,SAP C11,20015756,DEFOND ELECTRICAL INDUSTRIES/LIMITED,8.644296e+07,GTS,Purchased Components,defond electrical industries limited,d,defondelectricalindustrieslimited,Old


In [ ]:
merged_inp_old_flag_df = pd.merge(inp_old_flag_df, clean_main_df['Clean_Vendor_Name'], on=['Clean_Vendor_Name'], how='inner')
##merged_inp_old_flag_df.dropna(subset=['Clean_Vendor_Name','Vendor_name_Initials'],inplace=True)
merged_inp_old_flag_df.drop_duplicates(subset=['Vendor name'],inplace=True)
merged_inp_old_flag_df.reset_index(drop=True,inplace=True)

In [ ]:
merged_inp_old_flag_df.head()

,DataSource,Vendor number,Vendor name,Spend (USD),Segment,Primary Category,Clean_Vendor_Name,Vendor_name_Initials,crisp_vendor_name,Vendor_Flag
0,SAP C11,20010276,SAMSUNG SDI CO LTD,3.269116e+08,GTS,Purchased Components,samsung sdi,s,samsungsdi,Old
1,SAP E03,141048,"SAMSUNG SDI CO., LTD",8.548500e+05,GTS,Purchased Components,samsung sdi,s,samsungsdi,Old
2,CASS,KHNN-KUEHNE NAGEL SVCS LTD,KUEHNE NAGEL SVCS LTD,2.065791e+08,Undefined,Transportation,kuehne nagel svcs,k,kuehnenagelsvcs,Old
3,CASS,KHNN-KUEHNE NAGEL SVCS LTD,KUEHNE NAGEL SVCS LTD,1.990394e+07,Undefined,Transportation,kuehne nagel svcs,k,kuehnenagelsvcs,Old
4,SAP C11,20005014,CHANG TYPE INDUSTRIAL CO. LTD.,1.917764e+08,GTS,Finished Goods,chang type industrial,c,changtypeindustrial,Old


In [ ]:

merged_inp_old_flag_df.reset_index(drop=True,inplace=True)

if merged_inp_old_flag_df[merged_inp_old_flag_df.duplicated(subset='Vendor name')].shape[0]>0:
  merged_inp_old_flag_df[merged_inp_old_flag_df.drop_duplicates(subset='Vendor name')]


merged_inp_old_flag_df_VN_list = merged_inp_old_flag_df['Clean_Vendor_Name'].tolist()

In [ ]:
#inp_old_flag_df['noise_flag'] = inp_old_flag_df['Clean_Vendor_Name'].apply(lambda x: True if (re.findall("([^\x00-\x7F])+",x)) else False)
inp_old_flag_df_unfit= inp_old_flag_df[~inp_old_flag_df['Clean_Vendor_Name'].isin(merged_inp_old_flag_df_VN_list)]

In [ ]:
inp_old_flag_df_unfit.shape[0]

95

In [ ]:
inp_old_flag_df_unfit[inp_old_flag_df_unfit.duplicated(subset='Vendor name')].shape[0]

33

In [ ]:
inp_old_flag_df_unfit.to_excel(f'Input_old_unfit_data.xlsx',index=False,encoding='utf8')
files.download('Input_old_unfit_data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Check here whether the unfit record is present or not
#clean_main_df[clean_main_df.crisp_vendor_name.str.contains('yrc')].sample(5)
#c h robinson worldwide
#667 + 74

In [ ]:
#merged_inp_old_flag_df[merged_inp_old_flag_df.crisp_vendor_name.str.contains('aec')]

In [ ]:
no_of_fit_old_records = merged_inp_old_flag_df.shape[0]
#no_of_fit_old_records

In [ ]:
input_df.shape[0]

75591

In [ ]:
tot_no_of_null_inp_records = no_of_null_inp_records1 + no_of_null_inp_records2 
tot_no_of_null_inp_records

180

In [ ]:
no_of_unfit_old_records = inp_old_flag_df.shape[0] - no_of_fit_old_records
#no_of_unfit_old_records

In [ ]:
tot_no_of_clean_inp_records = input_df.shape[0]# this contains duplicate Vendon names but combination of all columns are unique
tot_no_of_unique_vendors = input_df['Vendor name'].nunique() 

In [ ]:
#input_df = input_df.sample(100)
input_df.reset_index(drop=True,inplace=True)


In [ ]:
if n==2:
    inp_df_New, inp_df_Old = [x for _, x in input_df.groupby(input_df['Vendor_Flag'] == 'Old')]
    print(inp_df_Old['Vendor name'].nunique())

    inp_ini_list = np.sort(inp_df_Old.Vendor_name_Initials.unique())
    grp_input_df_new=[]
    
    # Cleaning chamber:
    inp_df_New_output=inp_CleaningChamber(inp_df_New) #added new line after this line # this is one of the Dataframe
    inp_df_New_output.drop(columns=['Vendor_name_Initials','Clean_Vendor_Name'],axis=1,inplace=True)
    inp_df_New_output['Vendor_Name_cleansed'] = inp_df_New_output['Vendor_Name_cleansed'].apply(lambda x: x.upper())
    # write a condition that if there is no values in datasource or vendor number then  replace the same with nan
    
    inp_df_New_output['Manual Harmonized']=np.nan
    inp_df_New_output['Match Ratio']=np.nan
    inp_df_New_output['True Match Flag']=np.nan
    inp_df_New_output['Match_word_len'] = np.nan
    
    inp_df_New_output= inp_df_New_output[['DataSource','Vendor number','Vendor name','Vendor_Name_cleansed','Manual Harmonized','Vendor_Flag','Spend (USD)','Segment','Primary Category','Match Ratio','Match_word_len','True Match Flag']]
    
    inp_df_New_output.columns=['DataSource','Vendor number','Vendor name','Predicted Names/ Model Cleansed','Manual Harmonized','Vendor_Flag','Spend (USD)','Segment','Primary Category','Match Ratio','Match_word_len','True Match Flag']

    inp_df_New_output.reset_index(drop=True,inplace=True)
    
    
    # Model chamber:
    inp_ini_list = inp_ini_list.tolist()
    grp_input_df = bucket_df(inp_df_Old,inp_ini_list)
    #for i in range(len(grp_input_df)):
    #    if grp_input_df[i].Vendor_name_Initials.count()<1:  
    #        rem_ini = grp_input_df[i].Vendor_name_Initials.unique()[0]
    #        print('Removed Initials: ',rem_ini)
    #        inp_ini_list.remove(rem_ini)
    
    #for l in range(len(grp_input_df)):
    #    if grp_input_df[l].Vendor_name_Initials.values[0] in inp_ini_list: #changed from inp_ini_list[0] from inp_ini_list
    #        grp_input_df_new.append(grp_input_df[l])
    
    final_frame = []
    for i in tqdm(range(len(inp_ini_list))):
        for j in range(len(clean_ini_list)):
            if inp_ini_list[i]==clean_ini_list[j]:
                # we need to pass index ('j') to model defination. After this you can do global to name the DF's
                print('index value of clean_ini_list to be matched with inp_ini_list : ',j)
                print(inp_ini_list[i])
                print(clean_ini_list[j])                
                time_taken,grp_number,sgd_classifier=sgd_clf(j)
                print(f'[Info] Time taken to train group member "{grp_number}": {time_taken}')
                
                df_list = grp_input_df[i]['Clean_Vendor_Name'].values
                #print(df_list)
                frame=[]
                pred_st_time =  datetime.now()
                for k in tqdm(df_list):
                    predict_val = pd.DataFrame(sgd_classifier.predict(count_vect.transform([str(k)])))
                    frame.append(predict_val)
                inter_frame = pd.concat(frame).reset_index()
                inter_frame.drop('index',axis=1,inplace=True)
                frame_df_list= pd.DataFrame(df_list)
                row_val= pd.concat([frame_df_list, inter_frame], axis = 1) # try global here
                pred_end_time =  datetime.now()
                pred_time_taken = pred_end_time - pred_st_time
                pred_time_taken = time_util(pred_time_taken)
                print(f'[Info] Time taken to predict {grp_number}: {pred_time_taken}')
                print('-----------------------------------------------------------------------------------------------')
                final_frame.append(row_val)  
    final_output = pd.concat(final_frame)
    final_output.reset_index(inplace=True)
    final_output.drop('index',axis=1,inplace=True)
    final_output.columns=['Vendor_names','New_Supplier_Pred']
    final_output['New_Supplier_Pred'] = final_output['New_Supplier_Pred'].apply(lambda x: x.upper())
    final_output['New_Supplier_Pred'] = final_output['New_Supplier_Pred'].apply(lambda x: x.upper())
    clean_main_df.sort_values(by=['Clean_Vendor_Name'])
    
    merged_final = pd.merge(final_output,inp_df_Old,left_on='Vendor_names',right_on='Clean_Vendor_Name',how='left')
    merged_final.drop_duplicates(subset=['Vendor name'],inplace=True)
    merged_final.drop(columns=['Vendor_names','Vendor_name_Initials'],axis=1,inplace=True)
    merged_final = merged_final[['DataSource','Vendor number','Vendor name','New_Supplier_Pred','Clean_Vendor_Name','crisp_vendor_name','Vendor_Flag', 'Spend (USD)','Segment','Primary Category']]    
    merged_final =  pd.merge(merged_final,clean_main_df[['Clean_Vendor_Name','crisp_vendor_name','New Supplier']],left_on='crisp_vendor_name',right_on='crisp_vendor_name',how='left') # previously used 'Clean_Vendor_Name'
    merged_final.drop_duplicates(subset=['Vendor name'],inplace=True)
    #merged_final.drop(columns=['Clean_Vendor_Name','crisp_vendor_name'],axis=1,inplace=True)
    merged_final = merged_final[['DataSource','Vendor number','Vendor name','New_Supplier_Pred','New Supplier','Vendor_Flag','Spend (USD)','Segment','Primary Category']] 
    merged_final.columns=['DataSource','Vendor number','Vendor name','Model_pred_Supplier','Prev_Predicted_Supplier','Vendor_Flag','Spend (USD)','Segment','Primary Category']
    merged_final['Match_ratio']= merged_final.apply(lambda x: fuzz.ratio(str(x['Model_pred_Supplier']),str(x['Prev_Predicted_Supplier'])),axis=1)
    merged_final['Match_word_len'] = np.nan
    merged_final['True Match Flag']= ''
    merged_final.reset_index(drop=True,inplace=True)

    for i in tqdm(range(len(merged_final))):
        Match_wordLength(i)

    for i in tqdm(range(len(merged_final))):
      if (str(merged_final['Match_ratio'][i])=='100') & (merged_final['Match_word_len'][i] == 'Same Length'):
          merged_final['True Match Flag'].loc[i]= 'Yes'
      else:
          merged_final['True Match Flag'].loc[i]= 'No'

    merged_final.columns=['DataSource','Vendor number','Vendor name','Predicted Names/ Model Cleansed','Manual Harmonized','Vendor_Flag','Spend (USD)','Segment','Primary Category','Match Ratio','Match_word_len','True Match Flag']
    merged_final['DataSource']=merged_final['DataSource'].apply(ds_cleansing)
    merged_final= final_output_sup_name_Cleaning(merged_final,'Predicted Names/ Model Cleansed')
    merged_final= final_output_sup_name_Cleaning(merged_final,'Manual Harmonized')


else:
  # put logic when the output is needed for only OLD records and n==1 
  # write a elif condition that if n==1 and only New records are there then
    grp_input_df_new=[]
    print('Else section is running ...')
    inp_ini_list = np.sort(input_df.Vendor_name_Initials.unique())
    # here need to add in case n<1. Add same as above
    inp_ini_list = inp_ini_list.tolist()
    grp_input_df = bucket_df(input_df,inp_ini_list)
    #for i in range(len(grp_input_df)):
    #    if grp_input_df[i].Vendor_name_Initials.count()<=2:   
    #        rem_ini = grp_input_df[i].Vendor_name_Initials.unique()[0]
    #        print('Removed Initials: ',rem_ini)
    #        inp_ini_list.remove(rem_ini)
            
    #for l in range(len(grp_input_df)):
    #    if grp_input_df[l].Vendor_name_Initials.values[0] in inp_ini_list[0]:
    #        grp_input_df_new.append(grp_input_df[l])
    
    final_frame = []
    for i in tqdm(range(len(inp_ini_list))):
        for j in range(len(clean_ini_list)):
            if inp_ini_list[i]==clean_ini_list[j]:
                # we need to pass index ('j') to model defination. After this you can do global to name the DF's
                print(j)
                time_taken,grp_number,sgd_classifier=sgd_clf(j)
                print(f'[Info] Time taken to train group member "{grp_number}": {time_taken}')
                df_list = grp_input_df[i]['Clean_Vendor_Name'].values
                frame=[]
                pred_st_time =  datetime.now()
                for k in tqdm(df_list):
                    predict_val = pd.DataFrame(sgd_classifier.predict(count_vect.transform([str(k)])))
                    frame.append(predict_val)
                inter_frame = pd.concat(frame).reset_index()
                inter_frame.drop('index',axis=1,inplace=True)
                frame_df_list= pd.DataFrame(df_list)
                row_val= pd.concat([frame_df_list, inter_frame], axis = 1) # try global here
                pred_end_time =  datetime.now()
                pred_time_taken = pred_end_time - pred_st_time
                pred_time_taken = time_util(pred_time_taken)
                print(f'[Info] Time taken to predict {grp_number}: {pred_time_taken}')
                print('-----------------------------------------------------------------------------------------------')
                final_frame.append(row_val)  
    final_output = pd.concat(final_frame)
    final_output.reset_index(inplace=True)
    final_output.drop('index',axis=1,inplace=True)
    final_output.columns=['Vendor_names','New_Supplier_Pred']
    final_output['New_Supplier_Pred'] = final_output['New_Supplier_Pred'].apply(lambda x: x.upper())
    final_output['New_Supplier_Pred'] = final_output['New_Supplier_Pred'].apply(lambda x: x.upper())
    clean_main_df.sort_values(by=['Clean_Vendor_Name'])

    merged_final = pd.merge(final_output,inp_df_Old,left_on='Vendor_names',right_on='Clean_Vendor_Name',how='left')
    merged_final.drop_duplicates(subset=['Vendor name'],inplace=True)
    merged_final.drop(columns=['Vendor_names','Vendor_name_Initials'],axis=1,inplace=True)
    merged_final = merged_final[['DataSource','Vendor number','Vendor name','New_Supplier_Pred','Clean_Vendor_Name','crisp_vendor_name','Vendor_Flag']]    
    merged_final =  pd.merge(merged_final,clean_main_df[['Clean_Vendor_Name','New Supplier']],left_on='Clean_Vendor_Name',right_on='Clean_Vendor_Name',how='left')
    merged_final.drop_duplicates(subset=['Vendor name'],inplace=True)
    merged_final.drop(columns=['Clean_Vendor_Name','crisp_vendor_name'],axis=1,inplace=True)
    merged_final = merged_final[['DataSource','Vendor number','Vendor name','New_Supplier_Pred','New Supplier','Vendor_Flag']] 
    merged_final.columns=['DataSource','Vendor number','Vendor name','Model_pred_Supplier','Prev_Predicted_Supplier','Vendor_Flag']
    merged_final['Match_ratio']= merged_final.apply(lambda x: fuzz.ratio(str(x['Model_pred_Supplier']),str(x['Prev_Predicted_Supplier'])),axis=1)
    merged_final['Match_word_len'] = np.nan
    merged_final['True Match Flag']= ''
    merged_final.reset_index(drop=True,inplace=True)

    for i in tqdm(range(len(merged_final))):
        Match_wordLength(i)

    for i in tqdm(range(len(merged_final))):
      if (str(merged_final['Match_ratio'][i])=='100') & (merged_final['Match_word_len'][i] == 'Same Length'):
          merged_final['True Match Flag'].loc[i]= 'Yes'
      else:
          merged_final['True Match Flag'].loc[i]= 'No'

    merged_final.columns=['DataSource','Vendor number','Vendor name','Predicted Names/ Model Cleansed',
                                  'Manual Harmonized','Vendor_Flag','Match Ratio','Match_word_len','True Match Flag']


    merged_final['DataSource']=merged_final['DataSource'].apply(ds_cleansing)
    #merged_final= final_output_sup_name_Cleaning(merged_final)


    
    
    
    
    

42728


  0%|          | 0/36 [00:00<?, ?it/s]

index value of clean_ini_list to be matched with inp_ini_list :  0
0
0
[Info] Time taken to train group member "0": 00 00 secs


  0%|          | 0/5 [00:00<?, ?it/s]

[Info] Time taken to predict 0: 00 00 secs
-----------------------------------------------------------------------------------------------
index value of clean_ini_list to be matched with inp_ini_list :  1
1
1
[Info] Time taken to train group member "1": 00 00 secs


...

  0%|          | 0/1989 [00:00<?, ?it/s]

[Info] Time taken to predict n: 3 mins & 46 secs
-----------------------------------------------------------------------------------------------
index value of clean_ini_list to be matched with inp_ini_list :  24
o
o
[Info] Time taken to train group member "o": 00 02 secs


  0%|          | 0/1008 [00:00<?, ?it/s]

[Info] Time taken to predict o: 00 13 secs
-----------------------------------------------------------------------------------------------
index value of clean_ini_list to be matched with inp_ini_list :  25
p
p
[Info] Time taken to train group member "p": 00 24 secs


  0%|          | 0/3143 [00:00<?, ?it/s]

In [ ]:
#merged_final['DataSource']=merged_final['DataSource'].apply(ds_cleansing)
merged_final.head()

In [ ]:
no_of_dup_merged_records = merged_final[merged_final.duplicated(subset=['Vendor name'])].shape[0] # this will give duplicates if any gets generated
no_of_dup_merged_records

In [ ]:
non_dup_merged_records = merged_final.shape[0] - no_of_dup_merged_records
non_dup_merged_records

In [ ]:
if no_of_dup_merged_records>0:
  merged_final.drop_duplicates(subset=['Vendor name'],inplace=True) # Put if condition when no_of_dup_merged_records> 0, then do the drop 

In [ ]:
merged_final.shape

In [ ]:
# drop Nan from vendor names
merged_final.dropna(subset = ["Vendor name"],inplace=True)  # here also put condition for NA then do Na
merged_final.sort_values("Spend (USD)", ascending=False,inplace=True)
merged_final.reset_index(drop=True,inplace=True)

In [ ]:
#merged_final

In [ ]:
validation_file = merged_final.copy()
validation_file.sort_values("Predicted Names/ Model Cleansed", ascending=True,inplace=True)
validation_file = validation_file[['Predicted Names/ Model Cleansed', 'Manual Harmonized', 'Vendor_Flag','DataSource', 'Vendor number', 'Vendor name','Spend (USD)','Segment','Primary Category', 'Match Ratio', 'True Match Flag']]
validation_file.reset_index(drop=True,inplace=True)

In [ ]:
#validation_file[validation_file['Vendor number'] =='13090']

In [ ]:
validation_file['Spend (USD)'].dtype

In [ ]:
#float(2.065791e+08)+float(8.561588e+07)

In [ ]:
#inp_df_New_output

In [ ]:
inp_df_New_output['Spend (USD)'].dtype

In [ ]:
merged_validation_file  = pd.concat([validation_file, inp_df_New_output])
merged_validation_file.sort_values("Spend (USD)", ascending=False,inplace=True)
merged_validation_file.reset_index(inplace=True,drop=True)
final_validation_file = merged_validation_file.copy()
final_validation_file = final_validation_file[final_validation_file['True Match Flag'] != 'No']
#merged_validation_file = merged_validation_file.groupby(['Predicted Names/ Model Cleansed'],axis=0).apply(lambda x: x[x['True Match Flag'] == 'Yes' ]['Spend (USD)'].sum())
#merged_validation_file.drop(columns=['Predicted Names/ Model Cleansed','Spend (USD)','Match Ratio','True Match Flag','Match_word_len'],inplace=True)
#merged_validation_file.rename(columns = {'Spend (USD)':'Spend (USD)_1'}, inplace = True)

In [ ]:
merged_validation_file['Spend (USD)'].dtypes

In [ ]:
merged_validation_file['Spend (USD)']=merged_validation_file['Spend (USD)'].astype(float)

In [ ]:
merged_validation_file = merged_validation_file[merged_validation_file['True Match Flag'] != 'No']
merged_validation_file.reset_index(inplace=True,drop=True)

In [ ]:
merged_validation_file['Spend (USD)']=merged_validation_file['Spend (USD)'].astype(float)
merged_validation_file = merged_validation_file.assign(totalSpend = merged_validation_file['Spend (USD)']).groupby('Predicted Names/ Model Cleansed').agg({'totalSpend':sum})


In [ ]:
#merged_validation_file

In [ ]:
#merged_validation_file = merged_validation_file.assign(totalSpend = np.where(merged_validation_file['True Match Flag']!='No',merged_validation_file['Spend (USD)'],0)).groupby('Predicted Names/ Model Cleansed').agg({'totalSpend':sum})
merged_validation_file.sort_values("totalSpend", ascending=False,inplace=True)

In [ ]:
#merged_validation_file#[merged_validation_file['Predicted Names/ Model Cleansed'] == 'MEI LOCK & SECURITY']#['totalSpend'].dtypes

In [ ]:
merged_validation_file.reset_index(inplace=True)

In [ ]:
#merged_validation_file[merged_validation_file['Predicted Names/ Model Cleansed'] == 'MEI LOCK & SECURITY']

In [ ]:
merged_validation_file['totalSpend']=merged_validation_file['totalSpend'].astype(float)

In [ ]:
merged_validation_file['Spend (USD)_cumulative'] = merged_validation_file['totalSpend'].cumsum()

In [ ]:
#merged_validation_file

In [ ]:
merged_validation_file['Spend (USD)_cumulative']=merged_validation_file['Spend (USD)_cumulative'].astype(float)
merged_validation_file['totalSpend']=merged_validation_file['totalSpend'].astype(float)

In [ ]:
merged_validation_file['Cumulative Rolling %'] = (merged_validation_file['Spend (USD)_cumulative']  / merged_validation_file['totalSpend'].sum())*100
merged_validation_file['Supplier %'] = (merged_validation_file['totalSpend']  / merged_validation_file['totalSpend'].sum())*100

In [ ]:
#merged_validation_file

In [ ]:
merged_validation_file['Spend (USD)_cumulative'] = merged_validation_file['Spend (USD)_cumulative'].apply(lambda x: f'{x:.2f}')

merged_validation_file['Cumulative Rolling %'] = pd.to_numeric(merged_validation_file['Cumulative Rolling %'], downcast="float")
merged_validation_file['Cumulative Rolling %'] = merged_validation_file['Cumulative Rolling %'].apply(lambda x: f'{x:.2f}')

merged_validation_file['Supplier %'] = pd.to_numeric(merged_validation_file['Supplier %'], downcast="float")
merged_validation_file['Supplier %'] = merged_validation_file['Supplier %'].apply(lambda x: f'{x:.2f}')
merged_validation_file.reset_index(drop=True,inplace=True)

In [ ]:
#merged_validation_file

In [ ]:
merged_validation_file['Group'] = ''
merged_validation_file['Cumulative Rolling %'] = pd.to_numeric(merged_validation_file['Cumulative Rolling %'], downcast="float")
merged_validation_file.loc[merged_validation_file['Cumulative Rolling %']<=80.0, ['Group']] = 1
merged_validation_file.loc[merged_validation_file['Cumulative Rolling %']>80.0, ['Group']] = 2
merged_validation_file.loc[merged_validation_file['Cumulative Rolling %']>90, ['Group']] = 3
merged_validation_file.loc[merged_validation_file['Cumulative Rolling %']>95, ['Group']] = 4
merged_validation_file.loc[merged_validation_file['Cumulative Rolling %']>98, ['Group']] = 5

merged_validation_file['Cumulative Rolling %'] = merged_validation_file['Cumulative Rolling %'].apply(lambda x: f'{float(x):.2f}')
merged_validation_file['totalSpend'] = merged_validation_file['totalSpend'].apply(lambda x: f'{float(x):.2f}')


In [ ]:
merged_validation_file

In [ ]:
final_validation_file =  pd.merge(final_validation_file,merged_validation_file[['Predicted Names/ Model Cleansed','totalSpend','Supplier %','Spend (USD)_cumulative','Cumulative Rolling %','Group']],left_on='Predicted Names/ Model Cleansed',right_on='Predicted Names/ Model Cleansed',how='left')
final_validation_file.sort_values("Predicted Names/ Model Cleansed", ascending=True,inplace=True)
final_validation_file.drop(columns=['Match Ratio', 'True Match Flag',	'Match_word_len'],inplace=True)
final_validation_file['Spend (USD)'] = final_validation_file['Spend (USD)'].apply(lambda x: f'{x:.2f}')
final_validation_file = final_validation_file[['Predicted Names/ Model Cleansed', 'Manual Harmonized', 'Vendor_Flag',\
                                        'DataSource', 'Vendor number', 'Vendor name', 'Spend (USD)',\
                                        'totalSpend','Supplier %','Spend (USD)_cumulative', 'Cumulative Rolling %',\
                                        'Group','Segment','Primary Category']]
final_validation_file.reset_index(drop=True,inplace=True)


In [ ]:
final_validation_file.tail(5)

In [ ]:
#stop

In [ ]:
#((326911619.44+85615877.26+206579069.43)/1806555966.77)*100

In [ ]:
#merged_validation_file.sort_values(['Predicted Names/ Model Cleansed'], ascending=True,inplace=True)
#merged_validation_file
#merged_validation_file.sort_values(['Predicted Names/ Model Cleansed'], ascending=True,inplace=True)
#temp_validation_file
#temp_validation_file.reset_index(drop=True,inplace=True)
#temp_validation_file
#total_spend = validation_file['Spend (USD)'].sum()
#total_spend

#temp_validation_file = validation_file.groupby(['Predicted Names/ Model Cleansed'],axis=0).apply(lambda x: x[x['True Match Flag'] == 'Yes' ]).agg({'Spend (USD)':sum})
#temp_validation_file = validation_file.assign(totalSpend = np.where(validation_file['True Match Flag']=='Yes',validation_file['Spend (USD)'],0)).groupby('Predicted Names/ Model Cleansed').agg({'totalSpend':sum})

#temp_validation_file

#merged_validation_file  = pd.concat([validation_file, inp_df_New_output])
#merged_validation_file.sort_values("Vendor name", ascending=True,inplace=True)
#merged_validation_file['Spend (USD)_cumulative'] = merged_validation_file['Spend (USD)'].cumsum()
#merged_validation_file.sort_values("Spend (USD)_cumulative", ascending=False,inplace=True)
#merged_validation_file['Rolling %'] = (merged_validation_file['Spend (USD)_cumulative']  / merged_validation_file['Spend (USD)'].sum())*100

#merged_validation_file['Rolling %'] = pd.to_numeric(merged_validation_file['Rolling %'], downcast="float")
#merged_validation_file['Spend (USD)_cumulative'] = merged_validation_file['Spend (USD)_cumulative'].apply(lambda x: f'{x:.2f}')
#merged_validation_file['Rolling %'] = (merged_validation_file['Rolling %'].apply(lambda x: f'{x:.2f}'))
#merged_validation_file['Rolling %'] = pd.to_numeric(merged_validation_file['Rolling %'], downcast="float")
#merged_validation_file.reset_index(drop=True,inplace=True)


#merged_validation_file['Group'] = ''
#merged_validation_file.loc[merged_validation_file['Rolling %']<=80.0, ['Group']] = 1
#merged_validation_file.loc[merged_validation_file['Rolling %']>80.0, ['Group']] = 2
#merged_validation_file.loc[merged_validation_file['Rolling %']>90, ['Group']] = 3
#merged_validation_file.loc[merged_validation_file['Rolling %']>95, ['Group']] = 4
#merged_validation_file.loc[merged_validation_file['Rolling %']>98, ['Group']] = 5

#merged_validation_file.head(15)

In [ ]:
model_chamber_records = merged_final.shape[0]
model_chamber_records


In [ ]:
cleansed_records = inp_df_New_output.shape[0]
cleansed_records

# Accuracy:

In [ ]:
true_matched_records = merged_final[merged_final['True Match Flag'] == 'Yes'].shape[0]
true_matched_records

In [ ]:
model_accuracy = (true_matched_records/model_chamber_records)*100
model_accuracy

In [ ]:
date = datetime.now().strftime("%Y_%b_%d")

In [ ]:
final_validation_file.to_excel('Validation.xlsx',index=False,encoding='utf8')
files.download('Validation.xlsx')

In [ ]:
# Send the ouput prredicted file to ouput folder
#merged_final.to_excel(f'C:/Users/PXC0909/OneDrive - Stanley Black & Decker/Desktop/SBD/Vendor Harmonization/VH_Data/Predicted/merged_final_{date}.xlsx',index=False,encoding='utf_8_sig')
merged_final.to_excel('Prediction.xlsx',index=False,encoding='utf8')
files.download('Prediction.xlsx')

In [ ]:
# Send the new/ Ambiguous vendor output file to conformation folder
#inp_df_New_output.to_excel(f'C:/Users/PXC0909/OneDrive - Stanley Black & Decker/Desktop/SBD/Vendor Harmonization/VH_Data/Cleansed/Cleansed_Vendors_{date}.xlsx',index=False,encoding='utf_8_sig')
inp_df_New_output.to_excel(f'Cleansed_Vendors.xlsx',index=False,encoding='utf8')
files.download('Cleansed_Vendors.xlsx')

In [ ]:
#inp_df_New_output.head()

In [ ]:
merged_final.shape

In [ ]:
process_end_time =  datetime.now()

In [ ]:
process_time_taken = process_end_time - process_start_time
process_time_taken = time_util(process_time_taken)

In [ ]:
print(f'Time taken to execute complete script: {process_time_taken}')

In [ ]:
#mytext = 'The program is executed' #+str(process_time_taken)
#language = 'en'
#myobj = gTTS(text=mytext, lang=language, slow=False)
#myobj.save("time_taken.mp3")
#os.system("time_taken.mp3")

In [ ]:
# Code to get the full information on data

# Information of Raw master data:::
# No of master files used:
print('No of master files/ file used: ',no_of_files)

# Total no of records in master data (including all type of records)
print("Total no of records: ", length)

# Total no of null records (repeatative cleaning)
tot_no_of_null_records = no_of_null_records1 + no_of_null_records2 + no_of_null_records3 
print("Total no of Null records: ",tot_no_of_null_records)

# No of bad/uncleaned master data:
print(f"No of bad/filtered/uncleaned master data: ", no_of_noise_flags)

# Total no of dropped records
no_of_dropped_records = tot_no_of_null_records + no_of_noise_flags
print("Total no of records removed: ",no_of_dropped_records)

# Total no of clean records:
no_of_clean_records =  length - no_of_dropped_records
print("No of Clean records:", no_of_clean_records)

# Total no of unique bad/uncleaned:
tot_no_of_unique_dirty_records = no_of_unique_dirty_records
print("No of (non-duplicate) records to clean: ", tot_no_of_unique_dirty_records)

# Total no of master records in use
print("No of records used as Master data: ", tot_no_of_clean_records)

# Percentage of unique bad/uncleaned records
print("Percentage of Foreign/ Dirty records: ", per_irr)


In [ ]:
# Information of Input data:::
# No of Input files used
print('No of input files used: ', tot_files)
tot_files

# Total no of records in input data (including all type of records)
print('No of records in input data: ', no_of_raw_inp_records)

# No of Null records
tot_no_of_null_inp_records = no_of_null_inp_records1 + no_of_null_inp_records2 

print('Total no of null records to drop: ', tot_no_of_null_inp_records)

# Unique Flags:
print("Unique Flags are: ",unique_vendor_flag(n))

# Foreign counts:
if foreign_count>0:
  print('No of foreign records: ',foreign_count)

# No of new records:
no_of_new_records = new_records
print('No of "New" records: ',no_of_new_records)


# No of old records:
no_of_old_records = old_records
print('No of "Old" records: ',no_of_old_records)

# No of fit old records:
no_of_fit_old_records
#print('No of fit old records: ', no_of_fit_old_records)

# No of unfit old records:
no_of_unfit_old_records
#print('No of unfit old records: ', no_of_unfit_old_records)

# total no of clean input records
print('Total no of clean inp records: ',tot_no_of_clean_inp_records)

# total no of Unique vendors
print('Total no of Unique vendors: ',tot_no_of_unique_vendors)


# After Run:
# Total no of records as output from Model chamber:
print("No of records predicted: ", model_chamber_records)

# Total no of records as output from cleansing chamber:
print("No of records cleansed: ", cleansed_records)

# Model Accuracy:
print("Model accuracy: ",model_accuracy)

# End of Script